In [1]:
import numpy as np
'''
    对于一个无向图, 其连通分量可以通过做全图 BFS/DFS 得到.

    以下专注于解决:
    给定一个有向图图, 输出其全部的强联通分量(Strongly Connected Components, SCC).

    实现思路：采用 Kosaraju 算法，主要步骤包括：
        1. 对原图 G 取反(反向图)记为 G_r
        2. 对 G_r 进行 DFS, 记录节点完成时间, 按完成时间从小到大存储在 node_list 中
        2. 对原图, 按 node_list 的反向顺序(完成时间从大到小)再次进行 DFS
        3. 每次 DFS 遍历到的节点集合构成一个强连通分量

    复杂度分析:
        如果是按照 PPT 上的, 算法的时间复杂度是 O(|V|+|E|), 应该是默认使用了邻接矩阵

        下面的方法构建了邻接矩阵.
        在下面代码中, dfs 需要访问邻接矩阵, 因此每一次全局 dfs, 复杂度复杂度 O(|V|^2)
        整体的复杂度亦为 O(|V|^2)
'''

'\n    对于一个无向图, 其连通分量可以通过做全图 BFS/DFS 得到.\n\n    以下专注于解决:\n    给定一个有向图图, 输出其全部的强联通分量(Strongly Connected Components, SCC).\n\n    实现思路：采用 Kosaraju 算法，主要步骤包括：\n        1. 对原图 G 取反(反向图)记为 G_r\n        2. 对 G_r 进行 DFS, 记录节点完成时间, 按完成时间从小到大存储在 node_list 中\n        2. 对原图, 按 node_list 的反向顺序(完成时间从大到小)再次进行 DFS\n        3. 每次 DFS 遍历到的节点集合构成一个强连通分量\n\n    复杂度分析:\n        如果是按照 PPT 上的, 算法的时间复杂度是 O(|V|+|E|), 应该是默认使用了邻接矩阵\n\n        下面的方法构建了邻接矩阵.\n        在下面代码中, dfs 需要访问邻接矩阵, 因此每一次全局 dfs, 复杂度复杂度 O(|V|^2)\n        整体的复杂度亦为 O(|V|^2)\n'

In [ ]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int]])->np.ndarray:
    vertex_id=[i for i in range(vertexNum)]
    adjM=np.zeros(shape=(vertexNum, vertexNum))

    # 不用numpy库实现 adjM 创建: adjM=[[0 for _ in range(numCourses)] for _ in range(numCourses)]

    for (u, v) in edges:
        adjM[u, v]=1 # 有向无权图

    return adjM

In [3]:
def dfs_global(adjM:np.ndarray)->list[int]:
    """
        对图进行全局DFS, 记录节点的完成时间顺序
        参数: adjM: 邻接矩阵表示的图
        返回: node_list: 按完成时间排序的节点列表（先完成的在前）
    """

    verNum=adjM.shape[0]

    finish=np.zeros(verNum, dtype=np.int16)
    mark=np.zeros(verNum, dtype=np.int16)
    time=0

    node_list=[]

    def _dfs_visit_node(node_id):
        nonlocal time
        mark[node_id]=1

        time+=1

        for u in range(verNum):
            if adjM[node_id, u]==1 and mark[u]==0:
                _dfs_visit_node(u)
            
        time+=1
        finish[node_id]=time
        node_list.append(node_id)
        mark[node_id]=2

    for v in range(verNum):
        if mark[v]==0:
            _dfs_visit_node(v)

    return node_list

In [ ]:
def find_strongly_connected_components(adjM:np.ndarray)->list[set[int]]:
    """
    寻找有向图的所有强连通分量(SCC)
    参数: adjM: 邻接矩阵表示的有向图
    返回: 强连通分量的列表-strongly_connected_components_list, 每个元素为一个包含节点的集合
    """
    
    node_list=dfs_global(adjM.T) # 这里实际是要对原图的所有边取反向然后 DFS, 只需要对邻接矩阵取转置就可以
    visit_list=node_list[::-1]  # 按照上面生成的序列的反向遍历

    strongly_connected_components_list=[]

    verNum=adjM.shape[0]
    mark=np.zeros(verNum, dtype=np.int16)

    def _dfs_visit_node(node_id)->set[int]:
        
        strongly_connected_component=set()
        strongly_connected_component.add(node_id) # 结点自身在自己的强连通分量中
        mark[node_id]=1

        for u in range(verNum):
            if adjM[node_id, u]==1 and mark[u]==0:
                tmp_set=_dfs_visit_node(u)
                for u in tmp_set:
                    strongly_connected_component.add(u)

        mark[node_id]=2
        return strongly_connected_component

    for v in visit_list:
        if mark[v]==0:
            strongly_connected_components_list.append(_dfs_visit_node(v))

    return strongly_connected_components_list            
        

![Test Pic 1](test1.png)

In [5]:
if __name__=='__main__':
    # 测试例子如上, 来自图论教材<离散数学, 尹宝林著, 第三版>P173
    # 结点 a-m 下标为 0-12, 如图所示
    
    edges=[
        (0, 1), (1, 2), (2, 0), (2, 4), (4, 5), 
        (2, 5), (3, 4), (5, 6), (6, 5), (6, 7),
        (7, 6), (6, 8), (7, 10), (10, 8), (8, 9),
        (9, 10), (10, 9), (11, 10), (11, 12), (12, 11)]

    vertexNum=13

    adjM=build_adjM(vertexNum, edges)
    strongly_connected_components=find_strongly_connected_components(adjM)

    print(strongly_connected_components)

[{8, 9, 10}, {11, 12}, {5, 6, 7}, {4}, {3}, {0, 1, 2}]


![Test Pic 2](test2.png)

In [6]:
if __name__=='__main__':
    # 第二个测试例子如上, 来自PPT: Strongly Connected Components
    # 原始节点下标为 1-10, 统一减1 变为 0-9
    
    edges=[
        (0, 2), (2, 3), (3, 0), (1, 5), (5, 4), (4, 1),
        (0, 9), (9, 0), (3, 5), (2, 6), (7, 8), (8, 7),
        (7, 9), (8, 6)]

    vertexNum=10

    adjM=build_adjM(vertexNum, edges)
    strongly_connected_components=find_strongly_connected_components(adjM)

    # 原始节点下标为 1-10, 统一减1 变为 0-9, 输出的强联通分量也是原始下标-1的
    print(strongly_connected_components)

[{6}, {1, 4, 5}, {0, 9, 2, 3}, {8, 7}]
